In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader  
from sklearn.model_selection import train_test_split
import numpy as np
from pathlib import Path
from models import EMGConvNet



# === 1. Define Paths ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === 2. Load Data ===
X = np.load(X_path)   # shape: (5, 8, 230000)
y = np.load(y_path) 


def extract_windows(X, y, window_size=500, stride=250):
    X_windows, y_targets = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]  # label = pose at last time step
            X_windows.append(x_window)
            y_targets.append(y_target)

    return np.stack(X_windows), np.stack(y_targets)

Xw, yw = extract_windows(X, y)

# === 4. Split into train/test ===
X_train, X_val, y_train, y_val = train_test_split(Xw, yw, test_size=0.2, random_state=42)



class EMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = EMGDataset(X_train, y_train)
val_dataset = EMGDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

model = EMGConvNet(
    conv_layers_config=[(16, 5, 1), (32, 5, 2), (64, 3, 2)],
    fc_layers_config=[128, 64],
    conv_dropouts=[0.1, 0.2, 0.3],
    fc_dropouts=[0.4, 0.4],
    output_dim=51,
    verbose=False
)
model.build()

# === 7. Training ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def compute_rmse(y_pred, y_true):
    return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

for epoch in range(10):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)

    model.eval()
    val_loss = 0.0
    val_rmse = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            rmse = compute_rmse(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
            val_rmse += rmse.item() * X_batch.size(0)

    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    val_rmse /= len(val_loader.dataset)

    print(f"Epoch {epoch+1:2d} | Train MSE: {train_loss:.4f} | Val MSE: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}")

Epoch  1 | Train MSE: 277.9984 | Val MSE: 201.0369 | Val RMSE: 14.1196
Epoch  2 | Train MSE: 169.9809 | Val MSE: 103.7185 | Val RMSE: 10.1121
Epoch  3 | Train MSE: 116.1787 | Val MSE: 73.1282 | Val RMSE: 8.4807
Epoch  4 | Train MSE: 102.8346 | Val MSE: 65.7876 | Val RMSE: 8.0574
Epoch  5 | Train MSE: 95.5643 | Val MSE: 65.2475 | Val RMSE: 8.0138
Epoch  6 | Train MSE: 89.5256 | Val MSE: 87.2492 | Val RMSE: 9.2696
Epoch  7 | Train MSE: 86.3286 | Val MSE: 69.2005 | Val RMSE: 8.2529
Epoch  8 | Train MSE: 85.8954 | Val MSE: 68.9281 | Val RMSE: 8.2557
Epoch  9 | Train MSE: 83.5447 | Val MSE: 68.1814 | Val RMSE: 8.1949
Epoch 10 | Train MSE: 80.7980 | Val MSE: 62.1608 | Val RMSE: 7.8172
